# Vectorized Multiple Linear & Polynomial Regression

This week's notebook will be heavy on theory, with a lot of different mathematics coming together to solve a seemingly simple problem. It will however also introduce several of the most important concepts in all of machine learning; modelling noise in the data, model complexity, and the resulting bias-variance trade-off. In addition, we'll use the tools from linear algebra to write much more compact and efficient versions of the linear model. Expect this programming assignment to take longer than previous notebooks to figure out, but afterwards you should also have a much better perspective on what machine learning is about.

Let's dive right in. The seemingly simple problem for this week is another regression data set, consisting of 30 data points. As you've now had an introduction to matrices, we'll call this data set what it is; a $30 \times 2$ data matrix. Below is the code to load the data.


In [ ]:
from notebook_checker import start_checks
# Start automatic globals checks
%start_checks

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

# Load the data
data = np.loadtxt('nonlinear_data.csv', delimiter=',', dtype=float)

# Print the data matrix
print(data.shape)
print(data)

In this notebook we'll start using *Numpy*, which is the standard Python library for working with vectors and matrices. The matrix above is a Numpy array with 30 rows and 2 columns, so has a `.shape` of `(30, 2)`. In their basic form, Numpy matrices are very similar to lists of lists in Python, where each row is just a list, and the whole matrix is a list of these rows.

There are a few subtle differences, but for now the only thing you need to know is that indexing just works a little differently than with a list of list. If you want the first element from the first row, with a list of lists you would write `list_of_lists[0][0]`, whereas with a Numpy array you would write `numpy_array[0, 0]`. The main advantage of course, is now we can use Numpy's built-in functions to very quickly compute linear algebra operations.

## Nonlinear data

The first column in this matrix contains the $x$ values and the second column contains the corresponding $y$ values of our training data. This is exactly the same type of data set as we used last week, with one important exception: The relationship between $x$ and $y$ is *probably not linear*. The language here is a little careful, because it can be hard to see just by looking at the data what the right kind of model is. Let's plot the data, so you can take a better look and make up your own mind.

The code below first splits the data into a vector for $x$ and a vector for $y$. The notation `[:, 0, None]` selects the first column from the data, while keeping it a *column* vector, which in *Numpy* has the shape $30 \times 1$. The version `[:, 0]` would also still select the first column, but automatically convert it to an array with shape $30$, so just a 1-dimensional array, which is *always* interpreted as a *row* vector. This can be a little confusing at first, but it is good to remember that a $30 \times 1$, a $1 \times 30$ and a $30$ shape array are not the same thing in *Numpy*, even if they might all represent the same vector.

Having $x$ and $y$ as $30 \times 1$ *column* vectors will be useful for later steps, so we'll just split them like this here already. After splitting the data, we print the `.shape` attribute of the `x` array to check the resulting dimensions.

In [ ]:
# Slice out the column vectors x and y
x = data[:, 0, None]
y = data[:, 1, None]
print(x.shape)

# Plot the data
plt.figure(figsize=(12,9))
plt.plot(x, y, 'r.')
plt.show()


## Matrix vectorization

Even if the data does not look linear, a simple linear model is still a good place to start. In the theory videos, Andrew described the process and advantages of vectorizing linear regression using the dot product between each sample vector $\mathbf{x}$ and the vector of weights $\mathbf{w}$ to compute the model prediction $\hat{y}$:

$$\begin{aligned}
f_{\mathbf{w},b}(\mathbf{x}) &= x_1w_1 + x_2w_2 + \dots + x_nw_n + b\\
&= \mathbf{x} \cdot \mathbf{w} + b\\
&= \hat{y}\\
\end{aligned}$$

Vectorization is very common in machine learning, as it can greatly speed up the process of learning on massive data sets. However, most of the time this vectorization is actually done using matrices containing multiple samples together, instead of just a vector dot product for each sample. The linear algebra operation used for this is matrix multiplication, and it requires just a small extension from the earlier model, but the result will be much more general.

First lets define what a matrix of samples looks like:

$$ X = \left[\begin{array}{cccc}
x_1^{(1)} & x_2^{(1)} & x_3^{(1)} & \cdots & x_n^{(1)} \\ 
x_1^{(2)} & x_2^{(2)} & x_3^{(2)} & \cdots & x_n^{(2)} \\ 
x_1^{(3)} & x_2^{(3)} & x_3^{(3)} & \cdots & x_n^{(3)} \\ 
\vdots & \vdots & \vdots & \ddots & \vdots \\
x_1^{(m)} & x_2^{(m)} & x_3^{(m)} & \cdots & x_n^{(m)} \\ 
\end{array} \right]$$

Note that $X$ is a capital letter, as we're referring to a whole matrix now, instead of a vector. Each row still corresponds to a sample of data, but each column now corresponds to a different feature for those samples. So, in the housing prices example each row would a different house, and each column would be a different attribute, like size in square meters or the number of bathrooms.

If we want to compute predictions $\mathbf{\hat{y}}$ for each of those $m$ different samples, we end up with a **system of linear equations**:

$$\hat{y}^{(1)} = x_1^{(1)}w_1 + x_2^{(1)}w_2 + \dots + x_n^{(1)}w_n + b$$
$$\hat{y}^{(2)} = x_1^{(2)}w_1 + x_2^{(2)}w_2 + \dots + x_n^{(2)}w_n + b$$
$$\hat{y}^{(3)} = x_1^{(3)}w_1 + x_2^{(3)}w_2 + \dots + x_n^{(3)}w_n + b$$
$$\vdots$$
$$\hat{y}^{(m)} = x_1^{(m)}w_1 + x_2^{(m)}w_2 + \dots + x_n^{(m)}w_n + b$$

And we can actually represent this whole system of linear equations using a matrix multiplication and a vector addition:

$$ 
\left[\begin{array}{c} \hat{y}^{(1)} \\ \hat{y}^{(2)} \\ \hat{y}^{(3)} \\
\vdots \\ \hat{y}^{(m)} \end{array} \right]
=
\left[\begin{array}{cccc}
x_1^{(1)} & x_2^{(1)} & x_3^{(1)} & \cdots & x_n^{(1)} \\ 
x_1^{(2)} & x_2^{(2)} & x_3^{(2)} & \cdots & x_n^{(2)} \\ 
x_1^{(3)} & x_2^{(3)} & x_3^{(3)} & \cdots & x_n^{(3)} \\ 
\vdots & \vdots & \vdots & \ddots & \vdots \\
x_1^{(m)} & x_2^{(m)} & x_3^{(m)} & \cdots & x_n^{(m)} \\ 
\end{array} \right]
\left[\begin{array}{c} w_1 \\ w_2 \\ w_3 \\ \vdots \\ w_n \end{array} \right]
+
\left[\begin{array}{c} b \\ b \\ b \\ \vdots \\ b \end{array} \right]
$$

**Note:** Make sure you understand that this matrix multiplication and vector addition are in fact *identical* to the system of linear prediction equations described above. Try to write this out for a small example data matrix using 3 samples and 2 feature, and compute the resulting prediction vector for some values of $\mathbf{w}$ and $b$.

### Broadcasted addition

In linear algebra we can only do addition between matrices or vectors of the same dimension. However ,in Numpy operations can be *broadcasted*, meaning we could add a single scalar value to a vector and the scalar would be added to *each* of the elements in the vector separately. As mentioned, this is not strictly part of linear algebra, but it is very useful operation, so we'll introduce a bit of notation for it: $\oplus$ is a broadcasted element-wise addition between a vector/matrix and a scalar. In Numpy this operation can just be done using the regular `+`, and the element-wise broadcasting will actually be done automatically! This is one of the many advantages of using Numpy, which we'll see more of in the next module.

Using this new broadcasted addition, we can now write the whole matrix multiplication and addition as one simple expression:

$$\mathbf{\hat{y}} = X\mathbf{w} \oplus b$$

Here $X\mathbf{w}$ is matrix multiplication between the data matrix $X$ and the parameter vector $\mathbf{w}$, just like in the example above. Then the scalar bias term $b$ is added to resulting predictions for each of the samples using the broadcasted addition $\oplus$. This operation avoids having to construct a whole vector of $b$ to add to each sample prediction (like in the example above), and instead adds the bias $b$ to each sample prediction directly.

### Assignment 1a: Linear model

Implement the function `linear_model` below, which takes a data matrix `X`, a parameter vector `w`, and a bias term `b`, and returns the vector of predictions. Use the Numpy function [np.matmul](https://numpy.org/doc/stable/reference/generated/numpy.matmul.html) to compute the matrix multiplication operation. The broadcasted addition can just be done using the regular `+` when working with Numpy arrays. Note that you cannot test this function just yet, but we'll do that next in assignment 1b.


In [ ]:
def linear_model(X, w, b):
    # YOUR CODE HERE


One of the main advantages of vectorization is the speed of the operation, as discussed in the theory videos. Another important advantage is *generality*, as this definition works for any number of features $n$. If we change the number of features in our data set $X$, as long as we make sure the dimensions of $\mathbf{w}$ still matches, then we can actually use of all of the same equations and functions as before.

For the linear model the matrix multiplication $X\mathbf{w}$ has dimensions $(m,\ n) \; (n,\ 1) = (m,\ 1)$, where $m$ is the number of samples, and $n$ is the number of features. As long as the number of features in $X$ and the number of features in $\mathbf{w}$ match, this multiplication will compute a vector with all $m$ hypotheses in a single operation. We'll use this trick throughout the notebook, which means we can very easily change the number of features of the model.

As this method works for any number of features, it could also be used to implement univariate linear regression from the previous module, by just using just a single feature. In that case $n$ is equal to $1$, so $X$ will have dimensions $m \times 1$ (a single column of features) and $\mathbf{w}$ will have dimensions $1 \times 1$ (exactly equal to the scalar $w$ from the previous module).

Note that, in order for this matrix multiplication to work in Numpy, we'll need every vector to be a 2-dimensional matrix. This includes the $1 \times 1$ vector $\mathbf{w}$, which in regular linear algebra would always just be scalar. This is a quirk of how Numpy is built, but given the many advantages the library also brings, it is worth dealing with. We've already taken care of this for the vectors `x` and `y` in the previous cell, as they both should now be shape `(30, 1)`.

### Assignment 1b: Applying the model to the data

Now we can test the linear model by applying a simple univariate regression to the data set. We can just fill in some values for $\mathbf{w}$ and $b$ for now, as we'll work on learning the best parameters later.

Use the Numpy function [np.ones](https://numpy.org/doc/stable/reference/generated/numpy.ones.html) to create a new $1 \times 1$ array containing the value $1$ for the vector `w`. The `b` parameter should just be a scalar number, which can also have a starting value of $1$. The code below will apply your function and plot the resulting prediction for the data points.

Note that you can try different values for `b`, and also multiply `w` with different numbers, to see the effects on the predicted line in the plot.

In [ ]:
# Create starting values for w and b
# w = ...
# YOUR CODE HERE

# Compute the linear model
y_hat = linear_model(x, w, b)

# Plot the model results
plt.figure(figsize=(12,9))
plt.plot(x, y, 'r.')
plt.plot(x, y_hat)
plt.show()

In [ ]:
assert np.allclose(linear_model(x, np.array([[3]]), 5), np.array([[5.],[5.10344828],[5.20689655],[5.31034483],[5.4137931],[5.51724138],[5.62068966],[5.72413793],[5.82758621],[5.93103448],[6.03448276],[6.13793103],[6.24137931],[6.34482759],[6.44827586],[6.55172414],[6.65517241],[6.75862069],[6.86206897],[6.96551724],[7.06896552],[7.17241379],[7.27586207],[7.37931034],[7.48275862],[7.5862069],[7.68965517],[7.79310345],[7.89655172],[8.]])), 'Does not seem to work for the given data set. Does the plot above look correct?'
assert np.allclose(linear_model(np.array([[3, 5, 7, 11], [13, 17, 19, 23], [28, 31, 37, 41]]), np.array([[43], [47], [53], [59]]), 61), np.array([[1445],[3783],[7102]])), 'Does not work with multiple features. Are you using matrix multiplication to generalize your model?'
assert w.shape == (1,1), 'Starting value for w has incorrect dimensions'
assert np.ndim(b) == 0, 'Starting value for b appear to be matrix and not a scalar'
print('Solution seems correct!')

### Assignment 2: Cost function

The next part of the linear model will be writing the cost function:

$$ J(\mathbf{w}, b) = \frac{1}{2m} \sum_{i=1}^m (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})^2$$

As we're writing vectorized code, you should write the entire function `linear_cost` using just matrix and vector operations, so *without any loops*! Use your `linear_model` function to compute the vector $\mathbf{\hat{y}}$, which contains the predictions of the model $f_{\mathbf{w},b}$ for each sample $\mathbf{x}^{(i)}$. You can use [np.square](https://numpy.org/doc/stable/reference/generated/numpy.square.html) and [np.sum](https://numpy.org/doc/stable/reference/generated/numpy.sum.html) to implement rest of the cost function without loops. However, with some careful thought, it is actually possible to write the whole function with just pure linear algebra operations, which is good practice.

Vectorized code can be tricky to write, but the code will work for any number of features and will be much more efficient when working with large data sets. The most important thing is to build your equation step by step, so start by computing just the vector with all the predictions using `linear_model`, then compute the vector of differences with the actual y-values, and so on. At each step, check that the result is of the right dimensions by printing the `.shape` attribute, before moving to the next step.

If you do write the cost function using matrix multiplication, the result will actually be a $1 \times 1$ matrix, and not a scalar number. In pure linear algebra terms a $1 \times 1$ matrix is always a scalar by definition, but, as mentioned, these are not the same thing in *Numpy*. You can use the function [np.squeeze](https://numpy.org/doc/stable/reference/generated/numpy.squeeze.html) as a final step to convert this resulting $1 \times 1$ *Numpy* matrix to an actual scalar.

In [ ]:
def linear_cost(w, b, X, y):
    # YOUR CODE HERE


print(linear_cost(w, b, x, y))

In [ ]:
assert np.isclose(linear_cost(np.array([[1]]), 1, x, y), 263.709614), 'Result of cost function seems incorrect'
assert np.ndim(linear_cost(w, b, x, y)) == 0, 'Result seems to be a matrix and not a scalar. Did you use np.squeeze?'
print('Solution seems correct!')

### Assignment 3: Gradient for b

Now that we have the cost function defined, we can move on to the gradients. The gradient term for $b$ will be identical to the one in the previous module, as this parameter is still just a scalar:

$$\frac{\partial J}{\partial b} = \frac{1}{m} \sum_{i=1}^m (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})$$

Implement the function `b_gradient` without any loops by using your `linear_model` function and [np.sum](https://numpy.org/doc/stable/reference/generated/numpy.sum.html).

In [ ]:
def b_gradient(w, b, X, y):
    # YOUR CODE HERE


In [ ]:
assert np.isclose(b_gradient(np.array([[1]]), 1, x, y), -7.587254), 'Result for b gradient function seems incorrect'
assert np.ndim(b_gradient(w, b, x, y)) == 0, 'Result seems to be a matrix and not a scalar.'
print('Solution seems correct!')

### Assignment 4: Gradient for vector w

Next we'll implement the gradient for the vector $\mathbf{w}$, which is where the vectorized approach really shines. If we write this in a general way, the function can compute the gradient for *any* number of parameters we include in our model, without needing to change the code at all.

From the cost function we can now determine the partial derivatives for each of the $\mathbf{w}$ parameters, using the exact same derivative rules as last week. The only difference is that there are now $n$ partial derivatives, for each of the $n$ parameters. The resulting set of equations looks like this:

$$\frac{\partial J}{\partial w_1} = \frac{1}{m} \sum_{i=1}^m (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})x^{(i)}_1$$
$$\frac{\partial J}{\partial w_2} = \frac{1}{m} \sum_{i=1}^m (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})x^{(i)}_2$$
$$\frac{\partial J}{\partial w_3} = \frac{1}{m} \sum_{i=1}^m (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})x^{(i)}_3$$
$$\dots$$
$$\frac{\partial J}{\partial w_n} = \frac{1}{m} \sum_{i=1}^m (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})x^{(i)}_n$$


Make sure you understand how this relates to the partial derivative equation from last week, as they should seem very similar. All these partial derivatives can be combined into a single gradient vector, like so:

$$\frac{\partial J}{\partial \mathbf{w}} = \left[\begin{array}{c} \frac{\partial J}{\partial w_1} \\ \frac{\partial J}{\partial w_2} \\
\frac{\partial J}{\partial w_3} \\ \vdots \\ \frac{\partial J}{\partial w_n} \end{array} \right]$$

This is really just an $n \times 1$ column vector of all the partial derivatives. In fact, we can again use the tools of linear algebra to compute the whole vector as a couple of operations, without using any loops. Write the function `w_gradient` to construct this vector without loops.

Here you should use the same steps as when vectorizing the cost function, so start with computing the prediction vector and then incrementally add the next steps. Remember to inspect the `.shape` at each step, to check the result has the dimensions you would expect.

Any operation where you are doing multiplications of elements, and then summing them afterwards, should make you think of trying to do this with a matrix multiplication. It can be a lot easier to think about matrix multiplication in terms of the dimensions of the matrices going in and coming out, rather than to think of the specific values of each matrix element. In general, when using a matrix multiplication the shapes should be

$$(a,\ b) \; (b,\ c) = (a,\ c)$$

so the columns of the first matrix should match the rows of the second matrix. This matched axis is then the axis that will get multiplied and summed across. If you run into dimension errors while multiplying, check that both matrices you're multiplying match on the correct axis, and the output matrix is of the shape you expect. Note that [np.transpose](https://numpy.org/doc/stable/reference/generated/numpy.transpose.html) can also be used to get the transpose of a matrix.

As this operation should work for any number of samples $m$ and any number of features $n$, you should try to write down the matrix dimensions in these general terms too, when checking the shapes of a multiplication. The result for the `w_gradient` should be a column vector of partial derivatives with the same shape as $w$, so the final dimensions of the gradient vector should always be $n \times 1$. In the case of simple univariate linear regression, this is just a $1 \times 1$ vector.


In [ ]:
def w_gradient(w, b, X, y):
    # YOUR CODE HERE

    
print(w_gradient(w, b, x, y))

In [ ]:
assert np.isclose(w_gradient(np.array([[1]]), 1, x, y), -8.349297), 'Result of gradient function seems incorrect'
assert np.allclose(w_gradient(np.array([[43], [47], [53], [59]]), 61, np.array([[3, 5, 7, 11], [13, 17, 19, 23], [28, 31, 37, 41]]), np.array([[1400],[3700],[7100]])), np.array([[423.33333333],[566.],[655.33333333],[828.66666667]])), 'Function does not work correctly with multiple features'
print('Solution seems correct!')

### Assignment 5: Gradient Descent

With the gradient vector code done, we can start with the actual `gradient_descent` function. You can (and should) take inspiration from your gradient descent code from the previous module's notebook. Include checks for when the cost is diverging and when the algorithm has converged. The only real difference here is that the $\mathbf{w}$ parameter is now a vector, instead of just a scalar. The parameter $b$ is still a scalar, so the update for that can remain identical.

With the gradient for $\mathbf{w}$ already returned as column vector, doing the gradient descent update for all the parameters in $\mathbf{w}$ can actually be neatly done as a single operation. Note that Numpy arrays support all the standard vector operations from linear algebra, so scalar multiplication can just be done with `*`, and vector addition and subtraction work with the regular `+` and `-`.

Once the algorithm has either converged or diverged, the function should return a tuple containing both $\mathbf{w}$ and $b$, so we can use *tuple unpacking* when calling the function to retrieve these learned parameters.
 

In [ ]:
def gradient_descent(X, y, w, b, alpha, thres=10**-6):
    # YOUR CODE HERE
    

# Find the theta vector that minimizes the cost function
w_fit, b_fit = gradient_descent(x, y, w, b, 0.9)
print("\nFound the following values for w & b:")
print(w_fit, b_fit)

# Plot the model results
print("\nResulting in the following model:")
y_hat = linear_model(x, w_fit, b_fit)
plt.figure(figsize=(12,9))
plt.plot(x, y, 'r.')
plt.plot(x, y_hat)
plt.show()

## Noise

Intuitively, the model above probably doesn't look like a good fit, even though it *is* the best possible linear line according to the cost function. In this next part of the assignment, we're going to change gears a bit and try to get a better perspective on how *good* or *bad* the fit of a model is exactly. For this we'll extend the model to include the *noise* within the data, so we can be more specific about how much we expect the data to differ from our fitted line model.

Noise is _any (random) variation_ in the data that is not a part of the underlying pattern you are trying to model. There are many possible causes for noise, the most straightforward option is that whatever instrument or measurement that was used to collect the data is not perfectly accurate. Random variations in the data can just be a result of imprecise measurement, but more commonly there are actually just other factors that should be part of the data that weren't, or even *can't* be, directly measured. If you don't have data for a factor that *does* actually affect the target variable, then your model can never capture the relationship between this factor and the target variable, therefore the effect of this factor should be considered noise.

Let's say you're trying to predict housing prices in Amsterdam, and you start off with a regression model based just on the number of square meters. You can probably make a decent prediction for the price of a house using this model, but more features would make your model better. For instance, you could include distance to Dam square from the house as a second feature, and the predictions would probably become quite a bit more accurate. You keep adding more and more features about the exact details of each house, but then you realise the state of the economy and the housing market at the time of sale will probably also be a relevant factor. Now you could also start adding in all sorts of stock market indices like *AEX* points, at each time of sale, to try and capture this aspect. This is going to make your data harder to collect, and probably it will *still* be incomplete after this. This is the most common source of noise; factors that affect the target variable that you can't or won't directly measure. 

## Gaussian noise distribution

The target variable in regression is a continuous value, so any random noise added to the data should also come from a continuous distribution. By far the most common continuous probability distribution used to model data is the *Gaussian* distribution, also called the *Normal* distribution. It turns out many things in nature can be approximated with Gaussians and it is a relatively simple model to use.

For this model we'll assume that there is an underlying function $g$ that perfectly captures the relationship between the variables $x$ and $y$ using some function. However, after the model is computed, some random noise drawn from a Gaussian distribution is added to the function output, meaning the measured values for some particular $x^i$ and $y^i$ will not perfectly fit the pattern of $g$. The Gaussian distribution for this noise has a $0$ mean and a variance of $\sigma^2$:

$$y^{(i)} \sim g(\mathbf{x}^{(i)}) + \mathcal{N}(0, \sigma^2)$$

This means the noise is symmetrical around the *true* underlying function $g$ and the only parameter defining the amount of noise is $\sigma$, which indicates how "wide" the spread of noise is.

*One of the fundamental goals in machine learning is to try and learn a model function that matches the true function $g$ as closely as possible, without including the noise.*

### Assignment 6: Computing residuals and estimating sigma

Now, if we assume that the model function $f_{\mathbf{w},b}$ does indeed match $g$, then we can replace $g$ with $f_{\mathbf{w},b}$ and move it to the left hand side, to get:

$$y^{(i)} - f_{\mathbf{w},b}(\mathbf{x}^{(i)}) \sim \mathcal{N}(0, \sigma^2)$$

The left hand side is now what is called the residual of the model; whatever is still left that the model didn't capture:

$$r^{(i)} = y^{(i)} - f_{\mathbf{w},b}(\mathbf{x}^{(i)})$$

These residuals are _exactly_ the noise component of the data, and so are assumed to have a 0 mean. Estimating the standard deviation for the residual vector then becomes:

$$\sigma = \sqrt{\frac{1}{m-1} \sum_{i=1}^m (r^{(i)} - 0)^2}$$

Write a function `compute_residual_std_dev` to compute the vector of residuals, based on a `y` vector of target values and an `y_hat` vector of predictions. The function should return the standard deviation of this residuals vector, assuming a 0 mean. You should be able to borrow some code from the `linear_cost` function for this, as the computations are really quite similar... Use [np.sqrt](https://numpy.org/doc/stable/reference/generated/numpy.sqrt.html) for the square root computation. If you write this function using matrix multiplication, don't forget to use [np.squeeze](https://numpy.org/doc/stable/reference/generated/numpy.squeeze.html) as a final step to convert the resulting $1 \times 1$ matrix to an actual scalar.

In [ ]:
from scipy.stats import norm

def compute_residuals_std_dev(y, y_hat):
    # YOUR CODE HERE


# Compute values required for the plot
sigma = compute_residuals_std_dev(y, y_hat)
samps = np.linspace(-3.5*sigma, 3.5*sigma, 200)
probs = norm.pdf(samps, scale=sigma)

# Draw the Gaussian distribution
plt.rc('axes', labelsize=24)
plt.figure(figsize=(12,9))
plt.plot(samps, probs)
plt.plot(y - y_hat, np.zeros(y.shape), 'r.', alpha=0.3)

# Draw sigma bounds within the distribution
plt.plot([0,0], [0,norm.pdf(0, scale=sigma)], 'g')
plt.plot([-sigma,-sigma], [0,norm.pdf(sigma, scale=sigma)], 'y')
plt.plot([sigma,sigma], [0,norm.pdf(sigma, scale=sigma)], 'y')
plt.plot([-3*sigma,-3*sigma], [0,norm.pdf(3*sigma, scale=sigma)], 'm')
plt.plot([3*sigma,3*sigma], [0,norm.pdf(3*sigma, scale=sigma)], 'm')

# Label things nicely
plt.title('Gaussian model of the residuals', fontsize=32)
plt.xlabel('Y-axis spread of points around the fitted line')
plt.ylabel('Probability density')
plt.show()

In [ ]:
assert np.isclose(compute_residuals_std_dev(y, y_hat), 15.653720), 'Residuals std dev seems incorrect'
assert np.ndim(compute_residuals_std_dev(y, y_hat)) == 0, 'Std dev seems to be a matrix instead of scalar'
print('Solution seems correct!')

## Probability density model for the noise

The model plotted above indicates the spread of the individual data points on the y-axis, as compared to the fitted line, i.e. the residuals. The probability density function indicates how likely a value is to occur; the higher the probability density, the more likely the value. The mean residual value of 0 is therefore "most likely". This statement is not technically correct though, because the probability of any specific continuous value is always 0. We can never say anything about the probability of a specific value in continuous distribution, but we can be exact about *ranges* of values. For instance:

* With a probability of 0.6827 a point will have a residual between $-\sigma$ and $\sigma$, indicated by the two yellow lines
* With a probability of 0.9973 a point will have a residual between $-3\sigma$ and $3\sigma$, indicated by the two magenta lines

### 3D version of the model

Below is the code to create a slightly different version of this same probability density plot, but using 3 dimensions. No need to write any code for this, but I think the plot helps to better understand the model for the residuals.

The $x$ and $y$ axis for this plot correspond to the exact same $x$ and $y$ axis in the earlier fitted line plots. So the red dots are data points for $x$ and $y$ and the green line is the model prediction. For this part of the plot there is no $z$ dimension, and everything is projected flat at $z=0$.

The blue wire mesh does have a $z$ component, namely the probability density at a point $x,y$. This Gaussian distribution models the noise on the $y$ axis, around the fitted line. The mean of this Gaussian shifts with this fitted line, ensuring the noise is always centered around the line.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

# Create the 3D plot
ax = plt.figure(figsize=(16,12)).add_subplot(111, projection='3d')
xs = np.linspace(0, 1, 150)
ys = np.linspace(-75, 100, 150)

# Compute the probability density surface
xr, yr = np.meshgrid(xs, ys)
pdf_map = np.vectorize(lambda x,y,w,b,s: norm.pdf((x*w + b) - y, scale=s), excluded=(2, 3))
zr = pdf_map(np.ravel(xr), np.ravel(yr), w_fit, b_fit, sigma).reshape(xr.shape)

# Plot the points and lines
ax.plot_wireframe(xr, yr, zr)
ax.scatter(x, y, c='r')
ax.plot(np.squeeze(x), np.squeeze(y_hat), c='g')

# Label things
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('Probability density')
plt.show()

### Assignment 7: Plotting model certainty

For the next step, we're going to combine this theory and the previous two plots into a single 2D plot that should give a much clearer picture of the certainty of the learned model. Write the function `plot_model_certainty`, which takes 5 arguments:

* `x`, the vector of $x$ values of the data set.
* `y`, the vector of $y$ values of the data set .
* `std_dev`, the estimate for the standard deviation of the residuals comparing $y^i$ and $f_{\mathbf{w},b}(\mathbf{x}^i)$.
* `x_samples`, a vector of evenly spaced samples on the x-axis, used to show the model predictions at every point.
* `y_predicts`, a vector of computed predictions, corresponding to each of the `x_samples` values.

Write the function `plot_model_certainty` to create a plot with the following attributes:

* The original data points should be red dots in the plot.
* The predicted line for the learned model, in green.
* A yellow bounding line $1\sigma$ *above* the predicted line, following the same shape as the predicted line.
* A yellow bounding line $1\sigma$ *below* the predicted line, following the same shape as the predicted line.
* A magenta bounding line $3\sigma$ *above* the predicted line, following the same shape as the predicted line.
* A magenta bounding line $3\sigma$ *below* the predicted line, following the same shape as the predicted line.

The resulting plot should give a clear picture of how certain the model is that each of the data points fall in a certain range, adding *confidence bounds* to the predicted line. The probabilities for points falling between the yellow and magenta lines are the same as in the first Gaussian plot.


In [ ]:
def plot_model_certainty(x, y, std_dev, x_samples, y_predicts):
    plt.figure(figsize=(12,9))
    
    # YOUR CODE HERE
    

# Create a column vector of 100 evenly spaced x-axis samples
samps = np.expand_dims(np.linspace(0, 1, 100), axis=1)

# Compute the model predictions for all 100 samples
preds = linear_model(samps, w_fit, b_fit)

# Plot model certainty with confidence bounds
plot_model_certainty(x, y, sigma, samps, preds)

#### Sidenote: Maximum Likelihood Estimation

This Gaussian modelling is not just useful for adding confidence bounds to a model, but also has a more fundamental function too, which it seems a waste to not at least mention. We won't go into the details of showing it here, but exactly this Gaussian model for noise, can also be used to try and find the parameters $\mathbf{w}$ and $b$ that are *most likely* given the data. Formally, this is called the *Maximum Likelihood Estimator*. It turns out that finding the MLE according to this model of the noise, is actually **identical** to finding the $\mathbf{w}$ and $b$ that minimize the cost function. This bit of theory links the probabilistic perspective of the data together with minimizing the cost function, and gives a strong theoretical reason as to why this particular cost function is a good one to use.

### Assignment 8: Polynomial feature engineering

Now that we have a multiple linear regression model and a way to show the confidence bounds of the model, we can get started with actually improving those confidence bounds.

Adding some extra input features could be a good way to improve the model, as then there would be more input information for the model to learn from. For the housing prices example, we could include the number of bathrooms and the number of floors for each house, in addition to the area of the house in square meters. If we have the extra data to try this, adding new features to the multiple linear regression model is trivial.

Alternatively, we could create some new *nonlinear features* from the existing feature and add those to the model input. By expanding what inputs we provide to the model, we can learn more complex nonlinear functions from the same data set. Somewhat confusingly, the model itself is then still just multiple *linear* regression, as we've only really changed what the input features for that linear model are.

Adding new features from existing ones is called feature engineering and the most common engineered features used in regression are polynomial features. Polynomial features expand the input with a squared, cubed, etc. feature in order to learn polynomial functions of a higher degree. The more polynomial terms you add, the more flexible your model will be become, allowing it to fit the data better.

The original univariate vector $\mathbf{x}$ is defined as:

$$ \mathbf{x} = \left[\begin{array}{c}
x^{(1)}\\ 
x^{(2)}\\
x^{(3)}\\ 
\vdots\\
x^{(m)}\\ 
\end{array} \right]$$

where $x^{(1)}$ is the x-value of first sample in the data set. We can expand this to a input matrix $X$ with $n^{th}$ degree features like so:

$$ X = \left[\begin{array}{cccc}
x^{(1)} & {(x^{(1)})}^2 & {(x^{(1)})}^3 & \cdots & {(x^{(1)})}^n \\
x^{(2)} & {(x^{(2)})}^2 & {(x^{(2)})}^3 & \cdots & {(x^{(2)})}^n \\ 
x^{(3)} & {(x^{(3)})}^2 & {(x^{(3)})}^3 & \cdots & {(x^{(3)})}^n \\ 
\vdots & \vdots & \vdots & \ddots & \vdots \\
x^{(m)} & {(x^{(m)})}^2 & {(x^{(m)})}^3 & \cdots & {(x^{(m)})}^n \\ 
\end{array} \right]$$

The notation here is a bit confusing, because of how the indexing is done. Some examples for clarity:

* ${(x^{(1)})}^2$ is the square of the original first x-value $x^{(1)}$, i.e. $x^{(1)} \cdot x^{(1)}$
* ${(x^{(m)})}^3$ is the cube of the original last x-value $x^{(m)}$, i.e. $x^{(m)} \cdot x^{(m)} \cdot x^{(m)}$

Write the function `polynomial_features`, which takes a feature vector `x` and returns an $m \times n$ matrix of polynomial features, with the highest degree being `n`. Each of the higher degree vectors can be computed using [np.power](https://numpy.org/doc/stable/reference/generated/numpy.power.html) and incrementally added to the matrix using [np.hstack](https://numpy.org/doc/stable/reference/generated/numpy.hstack.html).

Note that the links to these Numpy documentation pages also contain some small examples for each function, which you can use to try these functions yourself. Building your solution step by step will be key here, so make sure to start with a small input and check the intermediate results at every step.

In [ ]:
def polynomial_features(x, n):
    # YOUR CODE HERE
    

X_third_degree = polynomial_features(x, 3)
print(X_third_degree)

### Assignment 9: Putting it all together

We now actually already have all the components to fit these higher degree polynomials, as we can just reuse all of the multiple linear regression code from before. Write the function `plot_poly_descent`, which takes a vector `x`, a vector `y`, a maximum `degree` for the model, and learning rate `alpha`, and does the following:

1. Create the polynomial feature matrix of `x` to the correct `degree`.
2. Create a starting value for $b$ and starting vector for $\mathbf{w}$ of the correct dimensions, containing all *ones*.
3. Apply gradient descent using the polynomial feature matrix, these starting $\mathbf{w}$ and $b$, and the learning rate `alpha`. Store the resulting best $\mathbf{w}$ and $b$.
4. Compute the prediction vector $\mathbf{\hat{y}}$ using the new learned $\mathbf{w}$ and $b$ parameters.
5. Compute the standard deviation of the residuals of this model.
6. Create a column vector of 100 sample values, evenly spaced between 0 and 1.
7. Compute the model prediction for each of these samples, using the same polynomial feature expansion and learned $\mathbf{w}$ and $b$ parameters.
8. Create a plot of the model, including the confidence bounds, using all these elements.


In [ ]:
def plot_poly_descent(x, y, degree, alpha):
    # YOUR CODE HERE
    


print("\nFitted line and confidence bounds for degree 3 polynomial")
plot_poly_descent(x, y, 3, 0.9)

print("\nFitted line and confidence bounds for degree 5 polynomial")
plot_poly_descent(x, y, 5, 0.9)

## The Normal equation

If everything worked correctly, you should see plots for the fitted lines using $3^{rd}$ and $5^{th}$ degree polynomials. You should now be able to create polynomials of *any* degree using this code and find the best fitting parameters for that degree. The downside of the gradient descent approach though, is that the learning rate probably needs to be set separately for each of the higher degree polynomials. Additionally, with some of the lower learning rates, it might take a very long time for the algorithm to converge.

Specifically for multiple linear regression, there is an alternative solution for learning the model parameters. For most models, gradient descent is the only option, but here we have another solution, called the *Normal equation*. This equation requires a bit more more linear algebra than has been introducted so far, and so the details are outside the scope of this course. However, all this equation really does is solve the *exact point* where the entire gradient vector is equal to zero, so it just finds minimum of the cost function.

The function `normal_fit` uses the Normal equation as implemented in the *scikit-learn* library. The function returns the best fitting parameters $\mathbf{w}$ and $b$ for a data matrix `X` and a vector of target values `y`.

### Assignment 10: Plotting the Normal fit

Write the function `plot_poly_fit`, which should do exactly the same thing as `plot_poly_descent`, but replace the gradient descent portion with `normal_fit`. This way the code does not depend on a specific learning rate, and finding the best parameters $\mathbf{w}$ and $b$ should be much faster this way.

In [ ]:
from sklearn.linear_model import LinearRegression

def normal_fit(X, y):
    lr = LinearRegression()
    lr.fit(X, y)
    # Extract the w and b parameters from the fitted model
    return lr.coef_.T, lr.intercept_

def plot_poly_fit(x, y, degree):
    # YOUR CODE HERE


print("\nFitted line and confidence bounds for degree 10 polynomial")
plot_poly_fit(x, y, 10)

print("\nFitted line and confidence bounds for degree 30 polynomial")
plot_poly_fit(x, y, 30)

## Validation data

The last portion of this assignment we will focus on answering the question:

* Which degree of polynomial fits this data the best?

Increasing the degree of the polynomial will always add more parameters to the model, as the dimensions of $\mathbf{w}$ must also increase. More parameters means the model is more "flexible", as there are more values to change and create the *perfect fit*. This means, as you increase the degree of the model, the cost on the training data will always decrease, and the confidence bounds will always move closer together.

So is the perfect model just when the line moves exactly through all the data points and the confidence bounds are completely on top of the fitted line? No, because of the noise in the data, perfectly fitting all the data points will probably not generalize very well to new points. You can actually see a clear example of this in the degree 30 polynomial plot above. Between the first and second point, and between the last point and the point before that, the function makes strange spikes, that don't correspond to any data point. This is the best way for the polynomial to exactly fit these specific points apparently, but it doesn't capture any part of the underlying trend *between* these two points.

This type of model is said to be overfitting. It is fitting the noise in the data and no longer approximating the true function $g$. _You can detect overfitting by testing with data points you didn't train on._

### Assignment 11: K-fold cross-validation

In the previous assignments we used a separate test set to better estimate the performance of a learned model on new data points. A validation set is very similar, but serves a slightly different purpose. It is also a portion of the data that *isn't* used when training, but it is instead used to select the best of several trained models. If you then, after selecting the best model using the validation set, want an objective assessment of the performance of this model on new data, you would use a testing set. In proper machine learning research, you would always use all 3; training set, validation set, and testing set. For this assignment we'll skip the testing portion and only do validation to select the best degree polynomial model.

Splitting the data even further into a training and validation set, especially for a data set this small, has some challenges. The bigger you make your training set, the more data to fit your model parameters, so the better the learned model will be. However, the bigger you make your validation set, the better the estimate of the performance of the learned model. A classic solution to this trade-off is doing what is called *k-fold cross-validation*. You shuffle the data and divide it into $k$ equal parts (or folds). Then you repeat the training and validation process $k$ times, each time using a different part as the validation set, and all the other $k-1$ parts as the training set.

A common option is using $k=5$, which means you get 5 different train and validation splits, each consisting of $80\%$ training and $20\%$ validation. You can then average the training and testing costs for each of the 5 iterations, giving a better overall estimate. As an added benefit, during this whole process, every data point also gets used as a validation point exactly once, again improving the reliability of the estimates.

The function `k_fold_validation` below has already been mostly completed. Take a look at the code provided to see if you can understand how the different folds are constructed. Complete the last portion of the function to actually compute the average training and validation costs for all the folds and return these averages.

In [ ]:
def k_fold_validation(X, y, k=5):
    # Stack X and y together, so they get shuffled in the same way
    D = np.hstack((X, y))
    # Shuffle the rows of the combined data matrix D
    np.random.shuffle(D)
    
    # Compute the size of each of the folds
    fold_size = int(round(D.shape[0] / k))
    
    avg_train_cost = 0 
    avg_val_cost = 0
    # For each of the K folds
    for i in range(k):
        # Slice out the i^th validation section of fold_size number of rows
        x_val = D[i*fold_size:(i+1)*fold_size, :-1]
        # Slice out the corresponding validation y (column) vector
        y_val = D[i*fold_size:(i+1)*fold_size, -1, None]
        
        # Slice out the sections before and after the validation section
        # Stack these back together to form the training data
        x_train = np.vstack((D[:i*fold_size, :-1], D[(i+1)*fold_size:, :-1]))
        # Do the same for the y training vector
        y_train = np.vstack((D[:i*fold_size, -1, None], D[(i+1)*fold_size:, -1, None]))
        
        # Find the best fitting theta for this training set using the Normal equation
        w, b = normal_fit(x_train, y_train)

        # YOUR CODE HERE
        

    return avg_train_cost, avg_val_cost

training_cost, validation_cost = k_fold_validation(X_third_degree, y, 5)
print("Average training cost:", training_cost)
print("Average validation cost:", validation_cost)

### Assignment 12: Model selection

Usually k-fold cross validation is enough to get a good estimate of the training and validation costs for a model. However, in this particular case the data set is *very* small, and for the next plot to make sense, we'll need really reliable estimates. The best way to do that is to just repeat k-fold cross validation a number of times, with the data being shuffled at random before every split into folds, and average those results again.

Write the function `compute_train_val_costs`, which estimates the training and validation costs for a range of different polynomial degree models between 1 and `max_degree`. The function should return 3 lists, one simple list of the used degrees, and two other lists of the same length; one containing the average training cost for each degree and one containing the average validation cost for each degree.

Your function should construct the appropriate polynomial feature matrix for each degree and then repeat the 5-fold cross-validation process `iters` number of times. Each resulting training and validation cost should be averaged to a single training and a single validation cost for every polynomial degree tried.

In [ ]:
def compute_train_val_costs(x, y, max_degree, iters):
    # Create list of degrees and two cost lists
    degrees = list(range(1, max_degree+1))
    train_costs =  []
    validation_costs = []
    
    for deg in degrees:
        # YOUR CODE HERE
        
    
    return degrees, train_costs, validation_costs


degrees, train_costs, val_costs = compute_train_val_costs(x, y, 10, 1000)

# Plot with all the results
plt.figure(figsize=(12,9))
plt.plot(degrees, train_costs, label='training cost')
plt.plot(degrees, val_costs, label='validation cost')
plt.title('Underfitting vs. Overfitting', fontsize=32)
plt.xlabel('Polynomial degree')
plt.ylabel('Cost')
plt.legend(fontsize=18)
plt.show()

## Underfitting and overfitting

Understanding the plot above is essential. The x-axis shows the degree of the polynomial model, which corresponds to the complexity of that model; the more parameters the model has the more complex the learned model can be. There are 3 distinct areas in this plot:

* Degree 1-3: Training cost is high and validation cost is high. This is called **underfitting**. Here the model does not have enough complexity to actually fit the underlying function $g$ (like with the original degree 1 model). This means the training cost can never get low and produce a good fit, because there is no good solution for a model that is too simple.
* Degree 7-10: Training cost is very low and validation cost is high. This is called **overfitting**. Here the model is too complex and too flexible, so it will start to actually fit the noise. With enough flexibility the model can fit *any* function, including the random pattern in the noise. This will create a very low training cost, but won't generalize well to other data points (like in the validation set), so the cost on the validation set becomes very high.
* Degree 4-6: Training cost is low and validation cost is low: A good fit. Degree 4 seems to fit best, having the lowest validation cost. Here the model is flexible enough to fit the true function $g$, but not so flexible as to also start fitting the noise. This is the ideal result for your model.

### Sidenote: Bias Variance trade-off

An underfit model is also called a *high bias* model and an overfit model is also called a *high variance* model. These terms come from statistics and are often used instead of overfitting and underfitting, so it is good to know them. In fact, it is possible to formally define the bias and the variance of the model, and show that the cost for some set of data points only has 3 components:

$$\frac{1}{2m} \sum_{i=1}^m (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})^2 = Bias^2 + Variance + \sigma^2$$

* The bias, corresponding to the amount of underfitting.
* The variance, corresponding to the amount of overfitting.
* $\sigma$, the standard deviation of the noise component.

The noise component of the cost cannot be reduced, as it is random, so actually *the only way to get a better model is to improve where you are in this underfitting-overfitting spectrum.*

### Assignment 13: Plotting the best fit

Use [np.argmin](https://numpy.org/doc/stable/reference/generated/numpy.argmin.html) to find the minimum index of the validation costs list. Then use this index to find the corresponding degree for this minimum. Print the polynomial degree with the lowest validation cost. Use `plot_poly_fit` to plot the polynomial model with this best fitting degree.

In [ ]:
# YOUR CODE HERE
